In [1]:
!pip install pandasql
!pip install kaggle

  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26819 sha256=9859c0b6cd3c25252851908fe7b169ffe091049f9467a9c8ecff0bb37ea44673
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [2]:
import pandas as pd
import numpy as np
import pandasql
import re
import nltk

In [3]:
nltk.download('universal_tagset')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Load raw dataset

Pulled and unzipped data from [kaggle](https://www.kaggle.com/kanaryayi/recipe-ingredients-and-reviews?select=reviews.csv) using Kaggle API. 

In [5]:
!mkdir /root/.kaggle/
!touch /root/.kaggle/
!touch ~/.kaggle/kaggle.json

api_token = {"username":"songyuyve","key":"aadc7d6cea1b4b9310d2b3f1534827ce"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d kanaryayi/recipe-ingredients-and-reviews -f recipes.csv
!kaggle datasets download -d kanaryayi/recipe-ingredients-and-reviews -f reviews.csv

  0% 0.00/3.74M [00:00<?, ?B/s]
100% 3.74M/3.74M [00:00<00:00, 61.9MB/s]
 90% 81.0M/89.9M [00:01<00:00, 40.2MB/s]
100% 89.9M/89.9M [00:01<00:00, 47.9MB/s]


In [7]:
!unzip recipes.csv.zip recipes.csv
!unzip reviews.csv.zip reviews.csv

Archive:  recipes.csv.zip
  inflating: recipes.csv             
Archive:  reviews.csv.zip
  inflating: reviews.csv             


# Import raw data

In [8]:
recipes_df = pd.read_csv('recipes.csv', sep=';',  encoding= 'unicode_escape' , quotechar="'")
reviews_df = pd.read_csv('reviews.csv', delimiter=';', encoding='unicode_escape' , quotechar="'")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
recipes_df.head(5)

,Recipe Name,RecipeID,Review Count,Recipe Photo,Author,Prepare Time,Cook Time,Total Time,Ingredients,Directions
0,Golden Crescent Rolls Recipe,7000,304,https://images.media-allrecipes.com/userphotos...,Mike A.,25 m,15 m,3 h 10 m,2 (.25 ounce) packages active dry yeast**3/4 c...,"Dissolve yeast in warm water.**Stir in sugar, ..."
1,Poppy Seed Bread with Glaze Recipe,7001,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,15 m,1 h,1 h 20 m,3 cups all-purpose flour**1 1/2 teaspoons salt...,Preheat oven to 350 degrees F (175 degrees C)....
2,Czech Christmas Hoska Recipe,7002,11,https://images.media-allrecipes.com/userphotos...,Carol Ponec-Nemec,1 h,45 m,5 h 30 m,1 (0.6 ounce) cake compressed fresh yeast**1/4...,Dissolve yeast in warm water.**In a large bowl...
3,Applesauce Bread I Recipe,7003,124,https://images.media-allrecipes.com/userphotos...,GAF55,10 m,1 h 20 m,1 h 30 m,3 cups all-purpose flour**3 eggs**2 cups white...,Preheat oven to 350 degrees F (175 degrees C)....
4,Raisin Brown Bread Recipe,7004,5,https://images.media-allrecipes.com/userphotos...,Tina,30 m,40 m,3 h 10 m,2 cups boiling water**1 cup rolled oats**2 tab...,"Combine boiling water, oats, shortening, brown..."


In [10]:
reviews_df.head(5)

,RecipeID,profileID,Rate,Comment
0,7000,675719,5,Oh Yay! I've never made any kind of bread fro...
1,7000,1478626,5,These rolls are wonderful. I had to bake them ...
2,7000,608663,5,This recipe was made for a group of 120 in a c...
3,7000,2785736,5,WOO HOO.. I did it with the help from my fello...
4,7000,594474,5,"These are great!! Not a croissant, but much li..."


# Prepare recipe data

Hours to minutes

In [11]:
def time_to_min(row):
  ''' Unify time columns to minutes
  '''
  # check contains number
  if not any(char.isdigit() for char in row):
    return None
  
  day, hour, minut = 0, 0, 0
  hour_ind, day_ind = 0, 0
  if 'd' in row:
    day = float(row[:row.find('d')].strip(' '))
    row = row[row.find('d')+1:]
  if 'h' in row:
    hour = float(row[:row.find('h')].strip(' '))
    row = row[row.find('h')+1:]
  if 'm' in row:
    minut = float(row[:row.find('m')].strip(' '))
  return hour*60+minut+day*1440

In [12]:
recipes_df['Prepare_Time'] = recipes_df['Prepare Time'].apply(time_to_min)
recipes_df['Cook_Time'] = recipes_df['Cook Time'].apply(time_to_min)
recipes_df['Total_Time'] = recipes_df['Total Time'].apply(time_to_min)

recipes_df.drop(['Total Time', 'Cook Time', 'Prepare Time'], axis=1, inplace=True)
recipes_df.head(5)

,Recipe Name,RecipeID,Review Count,Recipe Photo,Author,Ingredients,Directions,Prepare_Time,Cook_Time,Total_Time
0,Golden Crescent Rolls Recipe,7000,304,https://images.media-allrecipes.com/userphotos...,Mike A.,2 (.25 ounce) packages active dry yeast**3/4 c...,"Dissolve yeast in warm water.**Stir in sugar, ...",25.0,15.0,190.0
1,Poppy Seed Bread with Glaze Recipe,7001,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,3 cups all-purpose flour**1 1/2 teaspoons salt...,Preheat oven to 350 degrees F (175 degrees C)....,15.0,60.0,80.0
2,Czech Christmas Hoska Recipe,7002,11,https://images.media-allrecipes.com/userphotos...,Carol Ponec-Nemec,1 (0.6 ounce) cake compressed fresh yeast**1/4...,Dissolve yeast in warm water.**In a large bowl...,60.0,45.0,330.0
3,Applesauce Bread I Recipe,7003,124,https://images.media-allrecipes.com/userphotos...,GAF55,3 cups all-purpose flour**3 eggs**2 cups white...,Preheat oven to 350 degrees F (175 degrees C)....,10.0,80.0,90.0
4,Raisin Brown Bread Recipe,7004,5,https://images.media-allrecipes.com/userphotos...,Tina,2 cups boiling water**1 cup rolled oats**2 tab...,"Combine boiling water, oats, shortening, brown...",30.0,40.0,190.0


In [13]:
# Reset id to RecipeId
recipes_df.set_index(keys=['RecipeID'],inplace=True)
recipes_df.head(5)

,Recipe Name,Review Count,Recipe Photo,Author,Ingredients,Directions,Prepare_Time,Cook_Time,Total_Time
RecipeID,,,,,,,,,
7000,Golden Crescent Rolls Recipe,304,https://images.media-allrecipes.com/userphotos...,Mike A.,2 (.25 ounce) packages active dry yeast**3/4 c...,"Dissolve yeast in warm water.**Stir in sugar, ...",25.0,15.0,190.0
7001,Poppy Seed Bread with Glaze Recipe,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,3 cups all-purpose flour**1 1/2 teaspoons salt...,Preheat oven to 350 degrees F (175 degrees C)....,15.0,60.0,80.0
7002,Czech Christmas Hoska Recipe,11,https://images.media-allrecipes.com/userphotos...,Carol Ponec-Nemec,1 (0.6 ounce) cake compressed fresh yeast**1/4...,Dissolve yeast in warm water.**In a large bowl...,60.0,45.0,330.0
7003,Applesauce Bread I Recipe,124,https://images.media-allrecipes.com/userphotos...,GAF55,3 cups all-purpose flour**3 eggs**2 cups white...,Preheat oven to 350 degrees F (175 degrees C)....,10.0,80.0,90.0
7004,Raisin Brown Bread Recipe,5,https://images.media-allrecipes.com/userphotos...,Tina,2 cups boiling water**1 cup rolled oats**2 tab...,"Combine boiling water, oats, shortening, brown...",30.0,40.0,190.0


## Recipe missing value imputation and removal

In [14]:
recipes_df.isna().sum()

Recipe Name        0
Review Count       0
Recipe Photo       0
Author             1
Ingredients      838
Directions       844
Prepare_Time    6519
Cook_Time       8487
Total_Time      6183
dtype: int64

In [15]:
# Remove recipes without ingredients or directions
recipes_df.dropna(axis=0, how='any', subset=['Ingredients', 'Directions'], inplace=True)
recipes_df.isna().sum()

Recipe Name        0
Review Count       0
Recipe Photo       0
Author             0
Ingredients        0
Directions         0
Prepare_Time    5676
Cook_Time       7644
Total_Time      5340
dtype: int64

# Split and split ingredient table from recipe

In [16]:
recipes_df['Ingredients'][0].split('**')

['2 (.25 ounce) packages active dry yeast',
 '3/4 cup warm water (110 degrees F/45 degrees C)',
 '1/2 cup white sugar',
 '1 teaspoon salt',
 '2 eggs',
 '1/2 cup butter, room temperature',
 '4 cups all-purpose flour',
 '1/4 cup butter, softened']

In [17]:
Ingred = recipes_df['Ingredients']

In [18]:
# Unit set, only in Singular form
unit_set = ['cup','ounce','pound','teaspoon','quart','tablespoon','slice','package']
all_unit = "|".join(unit_set)

In [19]:
Ingred[4].split('**')

['2 cups boiling water',
 '1 cup rolled oats',
 '2 tablespoons shortening',
 '3 tablespoons brown sugar',
 '4 teaspoons salt',
 '1 cup molasses',
 '1/2 cup warm water (110 degrees F/45 degrees C)',
 '1 teaspoon white sugar',
 '2 tablespoons active dry yeast',
 '3 cups warm water (110 degrees F/45 degrees C)',
 '12 cups bread flour',
 '1 1/2 cups raisins',
 '1 1/2 teaspoons ground cinnamon']

In [20]:
extract_ingred(Ingred[0])

NameError: ignored

In [21]:
def extract_ingred(row):
  rows = row.split("**")
  res = []
  for row in rows:
    unit, quant, ing = None, 0, row
    # If contain () use units in ()
    if '(' in set(row) and row.find(')')!=len(row)-1:
      unit = row[row.find('(')+1:row.find(')')]
      quant = re.findall("\d+\.\d+|\d+|\d{2,}?",unit)
      if quant: 
        quant = float(quant[0])
      else:
        quant = 0
      measure = unit[-1:]
      ing = (" ".join(row[row.find(')')+1:].split()[1:]))
      row = row[row.find(')')+1:]
    else:

      # Match quantity
      num = re.findall("\d+\/[1-9][0-9]*|\d+|\d{2,}|\d{3,}?",row)
      frac = 0
      # Handle fractions
      if num:
        if (len(num)>1 or "/" in num[0]):
        # Fraction
          for n in num:
            if "/" not in n:
              quant += int(n)
            else: 
              frac = n
              ind_slash = frac.find('/')
              frac = int(frac[ind_slash-1])/int(frac[ind_slash+1])
              quant+= frac
              break
        else:
          quant = float(num[0])
      
      # Store quantity as decimal
      quant = quant+frac

      # Match unit
      unit = re.search(rf"{all_unit}",row)
      if unit:
        unit = unit.group()

      # Remove suffixes with , or ()
      if ',' in set(ing):
        ing = row[: row.find(',')]
      if '(' in set(ing):
        ing = row[: row.find('(')]
      ing = " ".join(re.findall(rf"(?!{all_unit})\b([a-z]+)?", ing)).strip()

      
    # Tokenize ingredients and remove verbs
    ing_set = nltk.word_tokenize(ing)
    tagged = nltk.pos_tag(ing_set, tagset = 'universal')
    ing = " ".join([word[0] for word in tagged if word[1]=='NOUN'])

    res.append((round(quant,2),unit,ing))
  return res

In [22]:
New_ingred = Ingred.apply(extract_ingred)

In [23]:
ingredient_df = pd.DataFrame(New_ingred.explode())

In [24]:
ingredient_df.head(5)
ingredient_df[['quantity','unit', 'ingredient']] = pd.DataFrame(ingredient_df['Ingredients'].tolist(),index=ingredient_df.index)
ingredient_df.drop(['Ingredients'], axis=1, inplace=True)

In [25]:
ingredient_df

,quantity,unit,ingredient
RecipeID,,,
7000,25.0,.25 ounce,yeast
7000,1.5,cup,water
7000,1.0,cup,sugar
7000,1.0,teaspoon,salt
7000,2.0,None,eggs
...,...,...,...
27594,12.0,12 inch,pizza crust
27594,2.0,cup,mozzarella cheese
27594,14.0,14 ounce,pizza sauce


In [26]:
unique_ingred = ingredient_df['ingredient'].unique()

In [27]:
unique_ingred_df = pd.DataFrame(unique_ingred)

In [28]:
unique_ingred_df

,0
0,yeast
1,water
2,sugar
3,salt
4,eggs
...,...
3795,pizza crust bread shell
3796,BUITONI® Pesto Sun-Dried Tomatoes
3797,onion alfalfa sprouts
3798,sugar replacement


In [29]:
recipes_df.head(5)

,Recipe Name,Review Count,Recipe Photo,Author,Ingredients,Directions,Prepare_Time,Cook_Time,Total_Time
RecipeID,,,,,,,,,
7000,Golden Crescent Rolls Recipe,304,https://images.media-allrecipes.com/userphotos...,Mike A.,2 (.25 ounce) packages active dry yeast**3/4 c...,"Dissolve yeast in warm water.**Stir in sugar, ...",25.0,15.0,190.0
7001,Poppy Seed Bread with Glaze Recipe,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,3 cups all-purpose flour**1 1/2 teaspoons salt...,Preheat oven to 350 degrees F (175 degrees C)....,15.0,60.0,80.0
7002,Czech Christmas Hoska Recipe,11,https://images.media-allrecipes.com/userphotos...,Carol Ponec-Nemec,1 (0.6 ounce) cake compressed fresh yeast**1/4...,Dissolve yeast in warm water.**In a large bowl...,60.0,45.0,330.0
7003,Applesauce Bread I Recipe,124,https://images.media-allrecipes.com/userphotos...,GAF55,3 cups all-purpose flour**3 eggs**2 cups white...,Preheat oven to 350 degrees F (175 degrees C)....,10.0,80.0,90.0
7004,Raisin Brown Bread Recipe,5,https://images.media-allrecipes.com/userphotos...,Tina,2 cups boiling water**1 cup rolled oats**2 tab...,"Combine boiling water, oats, shortening, brown...",30.0,40.0,190.0


# Clean reviews

In [30]:
reviews_df.isna().sum()

RecipeID       0
profileID    178
Rate         178
Comment      365
dtype: int64

In [31]:
# Drop rows without ratings, drop comment column
reviews_df.dropna(subset=['Rate'], axis=0, inplace=True)
reviews_df.drop(['Comment'], axis=1, inplace=True)

In [51]:
# reviews_df.RecipeID.nunique()
(reviews_df[reviews_df['RecipeID'].isin(diff)].index)

Int64Index([ 422953,  422954,  422955,  422956,  422957,  422958,  422959,
             422960,  422961,  422962,
            ...
            1578890, 1578891, 1578892, 1578893, 1578894, 1578895, 1578896,
            1578897, 1578898, 1578899],
           dtype='int64', length=139529)

In [32]:
reviews_df.isna().sum()

RecipeID     0
profileID    0
Rate         0
dtype: int64

In [54]:
# Check if all recipesID in review exists in recipes
review_id = set(reviews_df.RecipeID.unique())
recipe_id = set(recipes_df.index.unique())
diff = list(review_id - recipe_id)

# Drop mismatch from review table
reviews_df.drop(index=(reviews_df[reviews_df['RecipeID'].isin(diff)]).index, inplace=True)
len(reviews_df)

1439193

In [55]:
reviews_df.to_csv('reviews_cleaned.csv')
unique_ingred_df.to_csv('ingredient.csv')
recipes_df.to_csv('recipes_cleaned.csv')
ingredient_df.to_csv('ingredient_recipe.csv')

Next steps: 

1.   Check if recipeID in review exists in recipes
2.   Group by ingredient

